# **Preprocessing Functional MRI Data with the fMRIPrep BIDS App**


***

本教程将教你如何在 Windows、macOS 和 Linux 系统 上使用 fMRIPrep 对 BIDS 格式的 fMRI 数据进行标准化预处理。

我们将使用 Docker 运行 fMRIPrep，以确保跨平台兼容和最大程度的可重复性。

- **Step 1: 准备 BIDS 格式的数据集**
- **Step 2: 通过 Docker 运行fMRIPrep**
- **Step 3: FreeSurfer 许可证**
- **Step 4: 运行 fMRIPrep**
- **Step 5: 检查输出结果**



# Step 1 准备 BIDS 格式的数据集

### 输入数据：符合BIDS格式的数据集


```
Bids/
├── dataset_description.json
├── participants.tsv
└── sub-001/
    ├── anat/
    │   ├── sub-001_T1w.nii.gz
    │   └── sub-001_T1w.json
    └── func/
        ├── sub-001_task-rest_bold.nii.gz
        └── sub-001_task-rest_bold.json
```


# Step 2 通过 Docker 运行fMRIPrep

In [ ]:
# 下列命令在linux、macos、windows下均可执行
docker pull nipreps/fmriprep:latest

**注意：配置 Docker 镜像加速器**

在国内拉取 Docker 镜像时，如果出现如下错误：

```
docker: Error response from daemon: Get "https://registry-1.docker.io/v2/": read tcp 192.168.30.91:51536->44.208.254.194:443: read: connection reset by peer.
```

这表明 Docker 客户端在尝试访问 Docker Hub 或其他注册中心时出错，具体是无法获取注册中心的 API 响应。

> 🔗 参考：[知乎文章 - Docker 镜像加速](https://zhuanlan.zhihu.com/p/24228872523)


打开 Docker Desktop → Settings → Docker Engine，添加以下内容：

```json
{
  "registry-mirrors": [
    "https://docker-cf.registry.cyou",
    "https://mirror.aliyuncs.com",
    "https://mirror.baidubce.com",
    "https://docker.nju.edu.cn",
    "https://docker.mirrors.ustc.edu.cn",
    "https://mirrors.tuna.tsinghua.edu.cn"
  ]
}
```

点击 Apply & Restar。

之后在命令行/终端再次输入docker pull 的相关命令。

我们将使用Docker来运行 fMRIPrep。

此外，请下载Python。获取 Docker 和 Python 后，我们可以使用命令行下载一个包装器来运行 fMRIPrep。命令如下所示。

步骤 2. fMRIPrep 使用 FreeSurfer 来重建大脑的解剖表面及执行部分配准步骤。因此，fMRIPrep 需要 FreeSurfer 授权许可。要获取许可并运行 FreeSurfer，用户需免费注册，注册链接见此处。填写注册表单后，许可文件会发送到你提供的邮箱。请将该许可文件保存至计算机上。我们将许可文件保存为 /Users/franklinfeingold/freesurfer.txt。更多相关信息请参见此链接。

# Step 3: 运行 fMRIPrep 的被试级预处理分析

步骤 3. 现在可以调用 fmriprep-docker 了。下面展示了执行命令。命令解释如下：

第一个参数是 BIDS 数据集的输入路径；

第二部分是输出目录路径；

第三个参数是关键字 participant，表示执行被试级分析（具体详情及其他可能参数请参见 BIDS-Apps 规范；对于 fMRIPrep，当前仅支持 participant）；

最后提供被试标签（participant label），不包括 sub- 前缀，指定要运行的被试。

你也可以传递其他参数给 fMRIPrep。为简洁起见，建议将输入路径、输出路径和分析级别放在其他参数之前。

# Step 4: 检查输出结果和日志文件

现在，我们已经完成了对该数据集的 fMRIPrep 预处理！下一步是什么？
运行完 fMRIPrep 后，建议打开生成的可视化报告（例如，下方展示了 T1w 的脑掩模和脑组织分割、EPI 到 T1 的配准，以及 BOLD 总结部分的截图）。这些报告以 HTML 格式编写，可以在浏览器中查看（推荐使用 Chrome 或 Firefox）。报告将引导你了解预处理步骤，并突出展示 fMRIPrep 过程中所做的一些关键决策。
建议仔细检查每位被试的报告，以评估预处理结果的质量。
在每个报告的“Methods”章节，你会找到对具体处理流程的详细描述，方便直接复制到你的论文中，并包含所有相关参考文献。

<div style="overflow-x: auto; white-space: nowrap;">

| CSF      | WhiteMatter | GlobalSignal | stdDVARS | non-stdDVARS | vx-wisestdDVARS | FramewiseDisplacement | tCompCor00 | tCompCor01 | aCompCor00 | aCompCor01 | Cosine00 | NonSteadyStateOutlier00 | X          | Y         | Z         | RotX       | RotY        | RotZ        |
|----------|-------------|--------------|----------|---------------|------------------|------------------------|------------|------------|------------|------------|----------|--------------------------|------------|-----------|-----------|------------|-------------|-------------|
| 6551.282 | 6476.4653   | 9874.576     | n/a      | n/a           | n/a              | n/a                    | 0          | 0          | 0          | 0          | 0        | 1                        | 0          | 0         | 0         | 0          | 0           | 0           |
| 6484.729 | 6473.4890   | 9830.212     | 1.09047  | 52.78273      | 1.05944          | 0.13528                | 0.02631    | -0.06731   | -0.04241   | 0.02774    | 0.06652  | 0                        | -0.00997   | -0.03134  | -0.000003 | 0.00133    | -0.00038419 | -0.00016819 |
| 6441.534 | 6485.7256   | 9821.212     | 1.07520  | 52.04383      | 1.03822          | 0.12438                | -0.04048   | 0.03415    | 0.04621    | 0.00577    | 0.06650  | 0                        | -0.000026  | -0.00924  | 0.05497   | 0.000997   | -0.00019745 | -0.00039899 |
| 6401.115 | 6481.4287   | 9808.847     | 1.05444  | 51.03868      | 1.02468          | 0.14159                | 0.04160    | 0.04360    | 0.00488    | 0.05244    | 0.06651  | 0                        | -0.000033  | 0.05794   | 0.06928   | 0.00011    | 0           | -0.00027737 |
| 6428.275 | 6486.5215   | 9800.982     | 1.03302  | 50.00197      | 1.00802          | 0.05001                | 0.03914    | -0.04856   | -0.08489   | -0.06395   | 0.06650  | 0                        | -0.000028  | 0.06668   | 0.04715   | -0.00013   | 0           | -0.00013986 |

</div>
